In [7]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import keras

import matplotlib.pyplot as plt
import math as mt

from keras.models import Sequential
from keras.layers import Dense , Dropout , Flatten
from keras.layers import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras import backend as K
K.set_image_dim_ordering('th')

# Root directory of the project
ROOT_DIR = os.getcwd()


# fix random seed for reproducibility
seed = 7
np.random.seed(seed)


# training configuration
batch_size = 400
epochs = 150

In [ ]:
# get data
train = 